In [1]:
# import load_data function from loadData.py
from loadData import load_data

train_data, train_labels, test_data, test_labels = load_data()

Loading data
Data loaded


In [2]:
# run a forward pass on the network using test_data[0] and return the output
from runNetwork import runNetwork

prediction = runNetwork(test_data[0])
print(prediction)


2023-05-11 18:01:06.937787: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/user/Documents/GitHub/MLPlayground/MLenv/lib/python3.9/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/user/Documents/GitHub/MLPlayground/MLenv/lib/python3.9/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/user/Documents/GitHub/MLPlayground/MLenv/lib/python3.9/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/user/Documents/GitHub/MLPlayground/MLenv/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/user/Documents/GitHub/MLPlayground/MLenv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   